# ctypes

## 基本用法

`ctypes` 是一个方便 `Python` 调用本地已经编译好的外部库的模块

In [1]:
from ctypes import util, CDLL

## 标准 C 库

In [2]:
# 使用 util 来找到标准 C 库：
libc_name = util.find_library('c')
# on WINDOWS
print libc_name

msvcr90.dll


In [3]:
# 使用 CDLL 来加载 C 库：
libc = CDLL(libc_name)
print libc

<CDLL 'msvcr90.dll', handle 6d490000 at 4f82190>


In [4]:
# 调用printf函数：
libc.printf("%s, %d\n", "hello", 5)

9

这里显示的 `9` 是 `printf` 的返回值表示显示的字符串的长度（包括结尾的 `'\0'`），但是并没有显示结果，原因是 `printf` 函数默认是写在标准输出流上的，与 `IPython` 使用的输出流不一样，所以没有显示结果。

## C 数学库

In [5]:
libm_name = util.find_library('m')
print libm_name

msvcr90.dll


In [6]:
# 调用 atan2 函数
libm = CDLL(libm_name)
libm.atan2(1.0, 2.0)

ArgumentError: argument 1: <type 'exceptions.TypeError'>: Don't know how to convert parameter 1

In [7]:
from ctypes import c_double

libm.atan2.argtypes = [c_double, c_double]
libm.atan2.restype = c_double
libm.atan2(1.0, 2.0)

0.4636476090008061

In [8]:
# python中
from math import atan2
atan2(1.0,2.0)

0.4636476090008061

## Numpy 和 ctypes

```c
float _sum(float *vec, int len) {
    float sum = 0.0;
    int i;
    for (i = 0; i < len; i++) {
        sum += vec[i];
    }
    return sum
}
```

假设这个函数已经编译成动态链接库，那么我们可以这样调用：
```python
from ctypes import c_float, CDLL, c_int
from numpy import array, float32
from numpy.ctypeslib import ndpointer

x = array([1,2,3,4], dtype=float32)

lib = CDLL(<path>)

ptr = ndpointer(float32, ndim=1, flags='C')
lib._sum.argtypes = [ptr, c_int]
lib._sum.restype = c_float

result = lib._sum(x, len(x))
```